# Testing Model on Human Genome Sequences

This is the code used to apply the model tha twe trained to human genome sequences, in an effort to identify sequences that are good candidate attachment sites.

Code written by: Matt Durrant

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
from random import shuffle
from os.path import isfile
from Bio import SeqIO

# TensorFlow and tf.keras
import tensorflow as tf
import random
from tensorflow import keras
from keras.utils import to_categorical

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

# https://www.tensorflow.org/tutorials/keras/classification
print(tf.__version__)

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/Shared\ drives/CS230/datasets
import matplotlib.pyplot as plt

random.seed(42)

1.15.0
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/Shared drives/CS230/datasets


In [3]:
from keras.models import load_model

# Loads the model
model = load_model('triple.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 160, 4, 64)        640       
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 160, 4, 32)        18464     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 80, 2, 32)         0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 80, 2, 64)         18496     
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 80, 2, 64)         36928     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 40, 1, 64)         0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 40, 1, 64)       

In [0]:
# Functions to generate one-hot encodings for DNA sequences.

def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if char.upper()=="A":
            char_idx = 0
        elif char.upper()=="C":
            char_idx = 1
        elif char.upper()=="G":
            char_idx = 2
        elif char.upper()=="T":
            char_idx = 3
        elif char.upper()=="N":
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1
            
def one_hot_encode_along_channel_axis(sequence):
    sequence = ''.join(sequence)
    to_return = np.zeros((160,4), dtype=np.int8)
    padlength = 160 - len(sequence)
    padded = sequence + "N"*padlength
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=padded, one_hot_axis=1)
    return to_return

def one_hot_encode_y(y, dim):
  outy = np.zeros((1, dim))
  outy[0,y] = 1
  return(list(outy))

In [0]:
# Load human genome
fasta = {rec.id: rec for rec in SeqIO.parse('GRCh38_latest_genomic.fna', 'fasta')}

In [0]:
# Gets all 1 megabase regions from human genome.
def get_all_regions(fasta, regionlength=1000000):
  regions = []
  for rec in fasta:
    seq = str(fasta[rec].seq)
    for i in range(0, len(seq), regionlength):
      end = i + regionlength
      if end > len(seq):
        end = len(seq)
      regions.append((rec, i, end))
  return regions
  

In [15]:
# Gets regions and randomly shuffles them.
regions = get_all_regions(fasta)
shuffle(regions)
print(regions[:10])
print(len(regions))

[('NC_000003.12', 156000000, 157000000), ('NC_000001.11', 73000000, 74000000), ('NC_000010.11', 17000000, 18000000), ('NC_000002.12', 99000000, 100000000), ('NC_000011.10', 117000000, 118000000), ('NC_000018.10', 80000000, 80373285), ('NC_000004.12', 30000000, 31000000), ('NC_000022.11', 31000000, 32000000), ('NC_000022.11', 44000000, 45000000), ('NC_000003.12', 27000000, 28000000)]
3786


In [0]:
# Function to loop through a 1 megabase genomic region, extract sequences,
# make one-hot encodings, and apply model to make predictions.
def get_signif_attsites(fasta, recname, start, end, seqlen=106):
  print(recname, start, end)
  seqbatch = []
  rec = fasta[recname]
  for i in range(start, end, 10):
    
    if i + seqlen > len(rec.seq):
      break

    seq = str(rec.seq[i:i+seqlen]).upper()
    if seq.count("A") + seq.count("C") + seq.count("G")+seq.count("T") != len(seq):
      continue
    
    seqbatch.append((rec.id, i, seq))
    
  onehot = np.array([one_hot_encode_along_channel_axis(seq[-1]).reshape(160, 4, 1) for seq in seqbatch])
  if onehot.shape[0] == 0:
    return None
  predictions = model.predict(onehot)
  outlist = []
  for row, res in enumerate(seqbatch):
    name, pos, seq = res
    out = (name, pos, predictions[row, 0], predictions[row, 1], predictions[row, 2])
    outlist.append(out)
  outdf = pd.DataFrame(outlist)
  outdf.columns = ['contig', 'pos', 'attB_prob', 'attP_prob', 'neg_prob']
  return(outdf)
  

In [17]:
# Now start looping through regions, making predictions, and writing to file.
for contig, start, end in regions[:3]:
  outfile = "human_genome_att_predictions/" + contig + ':' + str(start) + '-' + str(end) + '.tsv'
  if isfile(outfile):
    continue
  preds = get_signif_attsites(fasta, contig, start, end)
  if preds is not None:
    preds.to_csv(outfile, sep='\t', index=False)

NC_000010.11 17000000 18000000


# That's it!